Compare performances of `onepbiter!` and `onebpiter_slow!`

In [97]:
include("../code/headers.jl")
using BenchmarkTools

In [98]:
n = 1000
m = 600
seed = 1234
algo = MS(maxiter = 100, Tmax=1, gamma=1e-3, sigma=1e-10, convergence=:messages, tol=-1.0)
lm1 = LossyModel(Val(2), n, m, randseed=seed)
lm2 = deepcopy(lm1)
independent = falses(n)
basis = lightbasis(lm1.fg, independent);

Slow version:

In [99]:
@btime solve!($lm1, $algo, randseed=$seed, oneiter! = $onebpiter_slow!, independent = $independent, basis= $basis);

LoadError: MethodError: no method matching onebpiter_slow!(::FactorGraphGF2, ::MS, ::OffsetArray{Float64,1,Array{Float64,1}}; fact_perm=[287, 499, 367, 49, 46, 225, 239, 161, 75, 286  …  481, 33, 191, 124, 312, 203, 159, 539, 234, 81])
Closest candidates are:
  onebpiter_slow!(::FactorGraphGF2, ::MS, ::Any; perm) at /home/crotti/sourcecoding/code/gfrbp.jl:165 got unsupported keyword argument "fact_perm"
  onebpiter_slow!(::FactorGraphGF2, ::MS) at /home/crotti/sourcecoding/code/gfrbp.jl:165 got unsupported keyword argument "fact_perm"

Fast version:

In [100]:
@btime solve!($lm2, $algo, randseed=$seed, oneiter! = $onebpiter!, independent = $independent, basis= $basis);

  6.096 ms (63471 allocations: 3.08 MiB)


In [75]:
@time solve!(lm1, algo, randseed=seed, oneiter! = onebpiter!, independent=independent, basis=basis)

BPResults{MS}
  converged: Bool false
  parity: Int64 4000
  distortion: Float64 0.6035
  trials: Int64 1
  iterations: Int64 100
  maxdiff: Array{Float64}((0,)) Float64[]
  codeword: BitArray{1}
  maxchange: Array{Float64}((100,)) [2.0000000003835035, 2.0079999998552234, 2.0080070002771517, 2.0100039917439303, 2.0120510881781537, 2.010033062651537, 2.0150702494915143, 2.0191256329619667, 1.0376123159066482, 1.039719860990926  …  0.09469521046434881, 0.07845655866844359, 0.10253490871056936, 0.11001493426124864, 0.0985716222381281, 0.11168469962817518, 0.12159889815134561, 0.09500483960264816, 0.09294032768438454, 0.09511341242353266]
